# CV Auto-Fill Backend Testing

This notebook demonstrates testing the backend functionality.

## Setup

Make sure you've installed the backend in development mode:
```bash
cd backend
pip install -e .
```

In [ ]:
# Standard imports
import sys
import os
from pathlib import Path

# Add parent directory to path (alternative to installing package)
backend_dir = Path().resolve().parent
if str(backend_dir) not in sys.path:
    sys.path.insert(0, str(backend_dir))
    print(f"Added {backend_dir} to Python path")

# Now import from app
from app.services.llm_service import LLMService
from app.services.pdf_service import PDFService

In [ ]:
# Test PDF to image conversion
import base64

# Load a sample PDF (you'll need to provide the path)
pdf_path = "test_data/sample_cv.pdf"  # Update this path

if os.path.exists(pdf_path):
    with open(pdf_path, "rb") as f:
        pdf_bytes = f.read()
    
    # Convert to images
    images = await PDFService.convert_pdf_to_images(pdf_bytes)
    print(f"Converted {len(images)} pages to images")
    print(f"First image size: {len(images[0])} bytes (base64)")
else:
    print(f"PDF file not found: {pdf_path}")

In [ ]:
# Test vision-based CV parsing
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv(backend_dir / ".env")

# Get API key from environment
groq_api_key = os.getenv("GROQ_API_KEY")

if groq_api_key and 'images' in locals():
    # Initialize LLM service with Groq
    llm_service = LLMService(api_key=groq_api_key, provider="groq")
    
    # Parse CV from images
    print("Parsing CV from images...")
    cv_data = await llm_service.parse_cv_from_images(images)
    
    # Display results
    import json
    print("\n=== Parsed CV Data ===")
    print(json.dumps(cv_data, indent=2))
else:
    if not groq_api_key:
        print("GROQ_API_KEY not found in .env file")
    if 'images' not in locals():
        print("Please run the PDF to images cell first")

In [ ]:
# Display the first page image (optional - requires IPython)
if 'images' in locals():
    from IPython.display import Image, display
    import base64
    
    # Decode and display first page
    img_bytes = base64.b64decode(images[0])
    display(Image(data=img_bytes))